In [ ]:
# ==========================================================
# Maestría en Ciencia y Análisis de Datos
# Universidad Mayor de San Andrés
# ----------------------------------------------------------
#   Modelos lineales y modelos lineales generalizados
# ----------------------------------------------------------
#        Rolando Gonzales Martinez, Agosto 2024
# ==========================================================
#        Modelo de Regresión Logistico (Logit)
# ==========================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf

# --------------------- Cargar datos ----------------------------------
url = "https://raw.githubusercontent.com/rogon666/UMSA/main/MLMLG/datos/empleados_balanceado.csv"
datos = pd.read_csv(url)

# Convertir la columna 'renuncia' a binaria (0 = No, 1 = Si)
datos['renuncia_binaria'] = np.where(datos['renuncia'] == 'Si', 1, 0)
datos['renuncia_binaria'] = datos['renuncia_binaria'].astype('category')

# Calcular el número de ceros y unos, y sus porcentajes
conteo_renuncias = datos['renuncia_binaria'].value_counts().reset_index()
conteo_renuncias.columns = ['renuncia_binaria', 'count']
conteo_renuncias['percentage'] = (conteo_renuncias['count'] / conteo_renuncias['count'].sum()) * 100
conteo_renuncias['renuncia_binaria'] = conteo_renuncias['renuncia_binaria'].map({0: 'No', 1: 'Si'})
print(datos.head())

In [ ]:
# Crear el gráfico
plt.figure(figsize=(3, 5))
sns.barplot(data=conteo_renuncias, x='renuncia_binaria', y='count', palette={'No': '#00BFC4', 'Si': '#F8766D'})
for index, row in conteo_renuncias.iterrows():
    plt.text(row.name, row['count'], f"{row['count']} ({row['percentage']:.1f}%)", ha='center', va='bottom', size=12)
plt.title('Distribución de Renuncias de Empleados', size=15)
plt.xlabel('Renuncia', size=12)
plt.ylabel('Número de Empleados', size=12)
plt.show()

In [ ]:
# Modelo logit:
X = datos[['edad', 'ingreso_mensual','distancia_al_trabajo'
           ,'num_empresas_trabajadas'
           ,'horas_extras'
           ,'incremento_salarial_porcentaje'
           ,'anios_en_la_empresa'
           ,'anios_en_el_puesto'
           ,'anios_desde_ultimo_ascenso'
           ,'anios_con_actual_supervisor'
          ]] 
y = datos[['renuncia_binaria']] 
# Añadir una constante para el intercepto del modelo
X = sm.add_constant(X)
# building the model and fitting the data 
logit = sm.Logit(y, X).fit() 
print(logit.summary())

In [ ]:
# Modelo logit:
X = datos[['edad', 'ingreso_mensual','distancia_al_trabajo'
           ,'num_empresas_trabajadas'
           ,'horas_extras'
           ,'incremento_salarial_porcentaje'
           ,'anios_en_la_empresa'
           ,'anios_en_el_puesto'
           ,'anios_desde_ultimo_ascenso'
           ,'anios_con_actual_supervisor'
          ]] 
y = datos[['renuncia_binaria']] 
# Añadir una constante para el intercepto del modelo
X = sm.add_constant(X)
# building the model and fitting the data 
logit = sm.Logit(y, X).fit(cov_type='HC0') 
print(logit.summary())

In [ ]:
# Modelo logit:
X_reducido = datos[['edad', 'ingreso_mensual','distancia_al_trabajo'
           ,'num_empresas_trabajadas'
           ,'horas_extras'
           ,'anios_en_la_empresa'
           ,'anios_en_el_puesto'
           ,'anios_desde_ultimo_ascenso'
           ,'anios_con_actual_supervisor'
          ]] 
y = datos[['renuncia_binaria']] 
# Añadir una constante para el intercepto del modelo
X_reducido = sm.add_constant(X_reducido)
# building the model and fitting the data 
logit_reducido = sm.Logit(y, X_reducido).fit(cov_type='HC0') 
print(logit_reducido.summary())

In [ ]:
# Calcular log-odds (los coeficientes ya están en términos de log-odds)
log_odds = logit_reducido.params
variables = log_odds.index

# Crear un DataFrame para ordenar y visualizar
log_odds_df = pd.DataFrame({'Variable': variables, 'Log Odds': log_odds})
log_odds_df = log_odds_df.set_index('Variable')

# Ordenar por valor absoluto de los log-odds para importancia
log_odds_df['Importancia'] = log_odds_df['Log Odds'].abs()
log_odds_df = log_odds_df.sort_values(by='Importancia', ascending=False)

# Excluir la constante del DataFrame
log_odds_df_no_const = log_odds_df.drop(index='const')

# Crear el gráfico sin la constante
plt.figure(figsize=(10, 6))
plt.barh(log_odds_df_no_const.index, log_odds_df_no_const['Log Odds'], color='skyblue')
plt.xlabel('Log Odds (Coeficiente)')
plt.title('Importancia de las Variables en el Modelo Logit en Términos de Log Odds')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Modelo Gompit (log-logístico)
from statsmodels.genmod.families.links import loglog
gompit_model = sm.GLM(y, X_reducido, family=sm.families.Binomial(link=loglog()))
result_gompit = gompit_model.fit()

# Mostrar el resumen del modelo
print(result_gompit.summary())

In [ ]:
# Comparacion de modelos:
logit_summary = logit_reducido.summary2().tables[0]
logit_metrics = {
    "Log-Likelihood": logit_reducido.llf,
    "AIC": logit_reducido.aic,
}
gompit_summary = result_gompit.summary2().tables[0]
gompit_metrics = {
    "Log-Likelihood": result_gompit.llf,
    "AIC": result_gompit.aic,
}

# Crear un DataFrame para comparar las métricas
comparison_df = pd.DataFrame({
    "Metric": ["Log-Likelihood", "AIC"],
    "Logit": [logit_metrics["Log-Likelihood"], logit_metrics["AIC"]],
    "Gompit": [gompit_metrics["Log-Likelihood"], gompit_metrics["AIC"]]
})
comparison_df